In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
categorical = ['ip','app','channel','device','os']

In [4]:
train_data = pd.read_feather('../data/data/models/train_data.feather', nthreads=15)
train_labels = train_data['is_attributed'].values
train_data = train_data.drop(['is_attributed','day','click_id'], axis=1)

In [5]:
valid_data = pd.read_feather('../data/data/models/valid_data.feather')
valid_labels = valid_data['is_attributed'].values
valid_data = valid_data.drop(['is_attributed','day','click_id'], axis=1)

In [6]:
train_matrix = lgb.Dataset(train_data.values, label=train_labels,feature_name=list(train_data.columns),
                        categorical_feature=categorical)
valid_matrix = lgb.Dataset(valid_data.values, label=valid_labels,feature_name=list(valid_data.columns),
                        categorical_feature=categorical)

In [7]:
booster = {}
booster['boosting_type'] = 'gbdt'
booster['objective'] = 'binary'
booster['learning_rate'] = 0.1
booster['num_leaves'] = 64
booster['max_depth'] = 5
booster['min_child_samples'] = 100
booster['max_bin'] = 100
booster['subsample'] = 0.7
booster['subsample_freq'] = 1
booster['colsample_bytree'] = 0.7
booster['min_child_weight'] = 0
booster['min_split_gain'] = 0
booster['nthread'] = 15
booster['verbose'] = 0
booster['scale_pos_weight'] = 99.7
booster['metric'] = 'auc'

In [8]:
params = {}
params['params'] = booster
params['train_set'] = train_matrix
params['valid_sets'] = [valid_matrix]
params['num_boost_round'] = 1000
params['early_stopping_rounds'] = 50
params['verbose_eval'] = 100

In [9]:
model = lgb.train(**params)

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 50 rounds.
[100]	valid_0's auc: 0.981495
[200]	valid_0's auc: 0.982127
[300]	valid_0's auc: 0.982436
[400]	valid_0's auc: 0.982525
[500]	valid_0's auc: 0.982611
Early stopping, best iteration is:
[528]	valid_0's auc: 0.982647


In [10]:
model.save_model('../data/data/models/lightgbm.model')

In [11]:
importance = model.feature_importance(importance_type='split')
importance = pd.DataFrame(importance, columns=['importance'])
importance['feature'] = list(train_data.columns)
importance['importance'] = importance['importance'] / importance['importance'].max()
importance = importance[['feature', 'importance']]
importance = importance.sort_values(by='importance', ascending=False)
importance.head(10)

,feature,importance
5,channel,1.000000
1,ip,0.885978
3,os,0.703542
2,app,0.698205
8,app_cnt,0.609413
20,next_click,0.568171
13,ip_app_cnt,0.324600
0,hour,0.315866
6,ip_cnt,0.304221
12,ip_device_cnt,0.287239
